### Laden benötigter Zusatzbibliotheken:

In [ ]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import plotly.express as px

**Importieren des eben erstellten Dokuments**

In [ ]:
df = pd.read_csv("authors-csv.csv", encoding="utf-8")
df

### Geolokation der Orte:

<div style="background-color: #FFDEAD; border-left: solid #FF7F50 4px; border-radius: 4px; padding: 0.7em;" ">
    <p style="margin: 0;"> <b> Hinweis: </b> Im Folgenden kann der Name des user_agents frei gewählt werden. Ideal ist ein Name, der wenig aussagend, aber nicht zu generisch ist, da die user_agents bei Überlastung ggf. nicht mehr funktionieren. Vermieden werden sollten daher Namen wie "test", möglich wäre bspw. "workshop_halle_test_vorname" oder Ähnliches.</p>
</div>

In [ ]:
# Initialisieren des Geolocators: 
geolocator = Nominatim(user_agent="hier_bitte_etwas_eintragen")

In [ ]:
# Funktion, um Longitude und Latitude zu ermitteln:
def get_lat_long(place):
    try:
        location = geolocator.geocode(place)
        if location:
            return location.latitude, location.longitude
        else:
            return None, None
    except GeocoderTimedOut:
        return None, None

In [ ]:
# Funktion auf der Spalte "place" ausführen, um neue Spalten "lat" und "long" zu ergänzen: 
df[['lat', 'long']] = df['Geburtsort'].apply(lambda x: pd.Series(get_lat_long(x)))

In [ ]:
df

In [ ]:
df = df.dropna(subset=['lat', 'long'])
df

In [ ]:
# Gruppieren bei "place" und zählen der Vorkommen der einzelnen Orte:
place_counts = df.groupby('Geburtsort').size().reset_index(name='count')
place_counts

In [ ]:
# Zusammenführen der Zählungen mit dem ursprünglichen Dataframe in einem neuen Dataframe "df_with_counts":
df_with_counts = df.merge(place_counts, on='Geburtsort')
df_with_counts

### Kartenvisualisierung erstellen:

In [ ]:
# Karte erstellen:
fig = px.scatter_mapbox(df_with_counts, 
                        lat="lat", 
                        lon="long", 
                        hover_name="Geburtsort", 
                        hover_data=["count"],
                        size="count",
                        color="count",
                        zoom=4,
                        height=600)

# Update des Layouts
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# Karte anzeigen: 
fig.show()

In [ ]:
# Karte bei Bedarf als HTML-Datei speichern: 
fig.write_html("places_map_plotly.html")